In [ ]:
# Librerías
import os
import json
import re

# Ruta de la carpeta donde están las fichas técnicas en texto
#CARPETA_FICHAS = os.path.join("..", "..", "data", "inputs", "1_data_acquisition", "crawler")
CARPETA_FICHAS = os.path.join("..", "..", "data", "inputs", "1_data_acquisition", "wrangler", "A.A.S._100_mg_COMPRIMIDOS.txt")

"""
# Expresiones regulares para identificar secciones clave
SECCIONES = {
    "indicaciones_terapeuticas": r"(?<=Indicaciones terapéuticas:)(.*?)(?=\n[A-Z])",
    "posologia": r"(?<=Posología:)(.*?)(?=\n[A-Z])",
    "contraindicaciones": r"(?<=Contraindicaciones:)(.*?)(?=\n[A-Z])",
    "advertencias_precauciones": r"(?<=Advertencias y precauciones especiales de empleo:)(.*?)(?=\n[A-Z])",
    "interacciones": r"(?<=Interacción con otros medicamentos y otras formas de interacción:)(.*?)(?=\n[A-Z])",
    "fertilidad_embarazo_lactancia": r"(?<=Fertilidad, embarazo y lactancia:)(.*?)(?=\n[A-Z])",
    "efectos_capacidad_conducir": r"(?<=Efectos sobre la capacidad para conducir y utilizar máquinas:)(.*?)(?=\n[A-Z])",
    "reacciones_adversas": r"(?<=Reacciones adversas:)(.*?)(?=\n[A-Z])",
    "codigo_atc": r"(?<=Código ATC:)(.*?)(?=\n[A-Z])",
    "fecha_revision": r"(?<=Fecha de revisión del texto:)(.*?)(?=\n[A-Z])",
    "excipientes": r"(?<=Lista de excipientes:)(.*?)(?=\n[A-Z])",
    "incompatibilidades": r"(?<=Incompatibilidades:)(.*?)(?=\n[A-Z])",
}

"""
SECCIONES = {
    "indicaciones_terapeuticas": r"(?i)indicaciones? terap[eé]uticas?[:\s]*(.*?)(?=\n[A-Z])",
    "posologia": r"(?i)posolog[ií]a\s*y\s*forma\s*de\s*administraci[oó]n?[:\s]*(.*?)(?=\n[A-Z])",
    "contraindicaciones": r"(?i)contraindicaciones?[:\s]*(.*?)(?=\n[A-Z])",
    "advertencias_precauciones": r"(?i)advertencias?\s*(?:y|e)\s*precauciones?\s*especiales?\s*de\s*empleo?[:\s]*(.*?)(?=\n[A-Z])",
    "interacciones": r"(?i)interacci[oó]n(?:es)?\s*con?\s*otros?\s*medicamentos?\s*(?:y|e)?\s*otras?\s*formas?\s*de\s*interacci[oó]n?[:\s]*(.*?)(?=\n[A-Z])",
    "fertilidad_embarazo_lactancia": r"(?i)fertilidad?,?\s*embarazo?\s*(?:y|e)?\s*lactancia?[:\s]*(.*?)(?=\n[A-Z])",
    "efectos_capacidad_conducir": r"(?i)efectos?\s*(?:sobre|en)?\s*la?\s*capacidad?(?:\s*para)?\s*conducir?\s*(?:y|e)?\s*utilizar?\s*m[aá]quinas?[:\s]*(.*?)(?=\n[A-Z])",
    "reacciones_adversas": r"(?i)reacciones?\s*adversas?[:\s]*(.*?)(?=\n[A-Z])",
    "codigo_atc": r"(?i)c[oó]digo\s*ATC:?[ \t]*(.*?)(?=\n[A-Z])",
    "fecha_revision": r"(?i)fecha\s*de\s*la\s*revisi[oó]n\s*del\s*texto:?[ \t]*(.*?)(?=\n[A-Z])",
    "excipientes": r"(?i)lista\s*de\s*excipientes:?[ \t]*(.*?)(?=\n[A-Z])",
    "incompatibilidades": r"(?i)incompatibilidades:?[ \t]*(.*?)(?=\n[A-Z])"
}

# Función para limpiar el texto de una sección
def parsear_ficha(texto, nombre_archivo):
    """Extrae la información clave de una ficha técnica de medicamento"""
    ficha_json = {
        "nombre": nombre_archivo
    }  # Nombre del archivo como nombre del medicamento

    # Extraer cada sección usando regex
    for key, regex in SECCIONES.items():
        match = re.search(regex, texto, re.DOTALL)
        ficha_json[key] = match.group(1).strip() if match else "No especificado"

    return ficha_json

# Función para procesar todas las fichas en una carpeta
def procesar_fichas(carpeta):
    """Lee todas las fichas técnicas en la carpeta y genera JSON para cada una"""
    fichas_procesadas = []

    # Bucle para procesar cada archivo en la carpeta
    if os.path.isdir(carpeta):
        print(f"Procesando fichas en {carpeta}...")
        for archivo in os.listdir(carpeta):
            if archivo.endswith(".txt"):  # Suponiendo que las fichas están en archivos .txt
                nombre_medicamento = os.path.splitext(archivo)[0]  # Nombre del archivo sin la extensión
                ruta_completa = os.path.join(carpeta, archivo)
                print(f"Procesando {ruta_completa}...")
                with open(ruta_completa, "r", encoding="utf-8") as f:
                    texto = f.read()
                    ficha_json = parsear_ficha(texto, nombre_medicamento)
                    fichas_procesadas.append(ficha_json)

    # Procesar un solo archivo
    else:
        print(f"Procesando ficha {carpeta}...")
        with open(carpeta, "r", encoding="utf-8") as f:
            texto = f.read()
            ficha_json = parsear_ficha(texto, os.path.splitext(os.path.basename(carpeta))[0])
            fichas_procesadas.append(ficha_json)

    return fichas_procesadas

# Procesar todas las fichas y guardar el JSON
fichas_json = procesar_fichas(CARPETA_FICHAS)
nombre_fichero = os.path.basename(CARPETA_FICHAS).replace(".txt", "")
print("Json generado:")
print(json.dumps(fichas_json, ensure_ascii=False, indent=4))
print("Procesamiento finalizado. JSON generado.")

# Guardar el JSON en un archivo
output_dir = "../../data/outputs/1_data_acquisition/wrangler" 
os.makedirs(output_dir, exist_ok=True)
if os.path.isdir(CARPETA_FICHAS):
    with open(os.path.join(output_dir, "fichas_tecnicas.json"), "w", encoding="utf-8") as f:
        json.dump(fichas_json, f, ensure_ascii=False, indent=4)
else:
    with open(os.path.join(output_dir, f"{nombre_fichero}.json"), "w", encoding="utf-8") as f:
        json.dump(fichas_json, f, ensure_ascii=False, indent=4)

Procesando ficha ../../data/inputs/1_data_acquisition/wrangler/A.A.S._100_mg_COMPRIMIDOS.txt...
Json generado:
[
    {
        "nombre": "A.A.S._100_mg_COMPRIMIDOS",
        "indicaciones_terapeuticas": "En base a su efecto antiagregante plaquetario está indicado en la profilaxis de: \n\n  Infarto  de  miocardio  o  reinfarto  de  miocardio  en  pacientes  con  angina  de  pecho  inestable  y  para",
        "posologia": "Posología: \n\n  Como inhibidor de la agregación plaquetaria: En la prevención de accidentes tromboembólicos (infarto,",
        "contraindicaciones": "No se debe administrar ácido acetilsalicílico en los siguientes casos: \n\n  Hipersensibilidad  al  principio  activo,  a  antiinflamatorios  no  esteroideos  o  a  tartrazina  (reacción",
        "advertencias_precauciones": "Dado el efecto antiagregante plaquetario del ácido acetilsalicílico (AAS), que aparece en dosis muy bajas y",
        "interacciones": "No especificado",
        "fertilidad_embarazo_lactancia